# STOCK MARKET -IndRNN

### Problem Description
The problem we are going to look at in this post is the stock market prices prediction problem.

This is a problem where, given a day, month, and a year, the task is to predict the price of next days stocks . The data ranges from 1996 to 2010,  with 4836 observations.

** Name of the Dataset **  “TATAMOTORS.xlsx'“

In [1]:
import numpy
import pandas
import math

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import os


from keras.callbacks import ModelCheckpoint
from ind_rnn import IndRNNCell, RNN

Using TensorFlow backend.


In [3]:
dataset = pandas.read_excel('TATAMOTORS.BO.xlsx', usecols=[0,4], skipfooter=3)
dataset.fillna(method='ffill', inplace=True)

plt.plot(dataset['Close'])
plt.show()

# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset sales-of-soaps.csv
dataframe = pandas.read_excel('TATAMOTORS.BO.xlsx', usecols=[4])
#dataframe = pandas.read_csv('international-airline-passengers.csv', usecols=[1], engine='python')
dataframe.fillna(method='ffill', inplace=True)

dataframe.head()
#print(type(dataframe.dtypes))

dataset = dataframe.values      ##   convert the data-frame to its Numpy-array
dataset = dataset.astype('float32')
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size

train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

forward_days = 10
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=40):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-forward_days+1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)


# reshape into X=t and Y=t+1

#40 is the number of days to consider in prediction
look_back = 40

"""
Doing a Train-Test Split
Train: 67%
Test: 33%
"""

trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
trainX.shape

# reshape input to be [samples, time steps, features]

trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))



,Date,Close
0,1996-03-06,93.709801
1,1996-03-07,94.188904
2,1996-03-08,95.626099
3,1996-03-09,96.967598
4,1996-03-10,100.033997


In [18]:
batch_size = 40
num_classes = 10
epochs = 10     # 200  
hidden_units = 64  # 128

learning_rate = 1e-3

In [19]:
#from keras.models import load_model

# cells = [IndRNNCell(hidden_units), IndRNNCell(hidden_units)]
cells = [IndRNNCell(4), IndRNNCell(4)]

from ind_rnn import IndRNNCell, RNN
from ind_rnn import IndRNN
#model = load_model('my_model.h5',  custom_objects={'IndRNNCell': IndRNNCell, 'IndRNN': IndRNN})

In [21]:

print('Evaluate IRNN...')
from ind_rnn import IndRNNCell, RNN
from ind_rnn import IndRNN

# create the IndRNN network
model = Sequential()
model.add(RNN(cells, input_shape=(look_back, 1), return_sequences=True))
model.add(IndRNN(32, recurrent_clip_min=-1, recurrent_clip_max=-1, dropout=0.0, recurrent_dropout=0.0, return_sequences=True))
model.add(RNN(cells, return_sequences=True))
model.add(IndRNN(32, recurrent_clip_min=-1, recurrent_clip_max=-1, dropout=0.0, recurrent_dropout=0.0, return_sequences=True))
model.add(RNN(cells, return_sequences=False))
model.add(Dense(1))

from keras import optimizers
#optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)


model.compile(loss='mean_squared_error', optimizer= 'adam', metrics=['accuracy'])


'''
Return sequences
Return sequences refer to return the hidden state a<t>. By default, the return_sequences is set to False in Keras RNN layers, and this means the RNN layer will only return the last hidden state output a<T>. The last hidden state output captures an abstract representation of the input sequence. In some case, it is all we need, such as a classification or regression model where the RNN is followed by the Dense layer(s) to generate logits for news topic classification or score for sentiment analysis, or in a generative model to produce the softmax probabilities for the next possible char.

In other cases, we need the full sequence as the output. Setting return_sequences to True is necessary.
'''

Evaluate IRNN...
Instructions for updating:
Colocations handled automatically by placer.


C:\Users\KIIT\Downloads\TimeSeries-1-master\TimeSeries-1-master\ind_rnn.py:143: UserWarning: IndRNNCell: Number of timesteps could not be determined. 
Defaulting to max clipping range of 1.0. 
If this model was trained using a specific timestep during training, inference may be wrong due to this default setting.
Please ensure that you use the same number of timesteps during training and evaluation
  warnings.warn("IndRNNCell: Number of timesteps could not be determined. \n"


In [22]:
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn_1 (RNN)                  (None, 40, 4)             356       
_________________________________________________________________
ind_rnn_1 (IndRNN)           (None, 40, 32)            192       
_________________________________________________________________
rnn_2 (RNN)                  (None, 40, 4)             356       
_________________________________________________________________
ind_rnn_2 (IndRNN)           (None, 40, 32)            192       
_________________________________________________________________
rnn_3 (RNN)                  (None, 4)                 356       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 745
Trainable params: 745
Non-trainable params: 0
_________________________________________________________________


In [24]:
#!pip install GraphViz 
import os
os.environ["PATH"] += os.pathsep + 'C:/ProgramData/Anaconda3/Library/bin/graphviz/'
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [26]:
%%time
# Fit the LSTM network
model.fit(trainX, trainY, epochs=200, batch_size=40, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/60
 - 4s - loss: 2.1404e-05 - acc: 3.1338e-04
Epoch 2/60
 - 2s - loss: 1.4416e-05 - acc: 3.1338e-04
Epoch 3/60
 - 2s - loss: 1.6087e-05 - acc: 3.1338e-04
Epoch 4/60
 - 3s - loss: 1.4451e-05 - acc: 3.1338e-04
Epoch 5/60
 - 2s - loss: 1.3802e-05 - acc: 3.1338e-04
Epoch 6/60
 - 2s - loss: 1.6244e-05 - acc: 3.1338e-04
Epoch 7/60
 - 2s - loss: 1.4505e-05 - acc: 3.1338e-04
Epoch 8/60
 - 2s - loss: 1.4319e-05 - acc: 3.1338e-04
Epoch 9/60
 - 2s - loss: 1.4321e-05 - acc: 3.1338e-04
Epoch 10/60
 - 2s - loss: 1.4925e-05 - acc: 3.1338e-04
Epoch 11/60
 - 2s - loss: 1.6470e-05 - acc: 3.1338e-04
Epoch 12/60
 - 2s - loss: 1.4164e-05 - acc: 3.1338e-04
Epoch 13/60
 - 2s - loss: 1.4424e-05 - acc: 3.1338e-04
Epoch 14/60
 - 2s - loss: 1.5358e-05 - acc: 3.1338e-04
Epoch 15/60
 - 2s - loss: 1.5190e-05 - acc: 3.1338e-04
Epoch 16/60
 - 2s - loss: 1.4250e-05 - acc: 3.1338e-04
Epoch 17/60
 - 2s - loss: 1.4947e-05 - acc: 3.1338e-04
Epoch 18/60
 - 2s - loss: 

Once the model is fit, we can estimate the performance of the model on the train and test datasets.

In [1]:
scores = model.evaluate(trainX, trainY, verbose=0)


NameError: name 'model' is not defined

In [2]:
from keras.models import load_model

print(scores)
print(model.metrics_names)

Using TensorFlow backend.


NameError: name 'scores' is not defined

In [30]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

***Note that we invert the predictions before calculating error scores 
to ensure that performance is reported in the same units as the original data (stock prices).***

In [31]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [32]:
# calculate root mean squared error
trainScore = mean_squared_error(trainY[0], trainPredict[:,0])
print('Train Score: %.2f MSE' % (trainScore))
testScore = mean_squared_error(testY[0], testPredict[:,0])
print('Test Score: %.2f MSE' % (testScore))

Train Score: 6.69 MSE
Test Score: 46.25 MSE


In [33]:
## LSTM   - Train Score: 23.70 RMSE Test Score: 58.87 RMSE (LSTM - 4)
## IndRNN - Train Score: 24.05 RMSE Test Score: 52.45 RMSE [Cell = 4,4]
## IndRNN - Train Score: 23.99 RMSE Test Score: 51.99 RMSE [Cell = 3,3]
## IndRnn - Train Score: 23.25 RMSE Test Score: 49.35 RMSE [Cell = 3,3] Look-back=6

Mean Squared Error:  0    46.245119
dtype: float64
Mean Absolute Error  0    4.923615
dtype: float64


Because of how the dataset was prepared, we must shift the predictions so that they align on the x-axis with the original dataset. Once prepared, the data is plotted, showing the original dataset in blue, the predictions for the training dataset in green, and the predictions on the unseen test dataset in black.

In [35]:
datedate = pandas.read_excel('TATAMOTORS.BO.xlsx', usecols=[0], skipfooter=3)

# plot baseline and predictions
plt.figure(figsize=(20,10))
plt.grid()
plt.plot(datedate.tail(1547),testPredict, label='Predicted Value', color='red')
plt.plot(datedate.tail(1547),testY[0], label='Real Value', color='blue')
plt.xlabel('Date', fontsize=25)
plt.ylabel('Price in USD', fontsize=25)
plt.xticks(fontsize=17)
plt.yticks(fontsize=17)
plt.legend()

plt.show()

ValueError: could not broadcast input array from shape (1547,1) into shape (1563,1)